#Mount Google Drive

This will allow the Colab machine to access Google Drive folders by mounting the drive on the machine. You will be asked to copy and paste an authentication code.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


# Change directory to allow imports


In [2]:
import os
os.chdir("/content/gdrive/My Drive/CMPT 419 720 Project/")
!ls

'=2.0.1'
 BoxDist24_GT
 BoxDist24_TrackRCNN_Training
 BoxDist31_GT
 BoxDist31_TrackRCNN_Training
 cmpt419_mono_dept_esitmation_poster_v5.pdf
'CMPT419 Project Report.gdoc'
 FairMOT
 FairMOT.ipynb
 MDE_Experiments
'Milestone Report 1.gdoc'
'Milestone Report 2.gdoc'
 MonoDepthEstimation_v0.1
 output
 poster.gdoc
 References
 sceneflow
'synthetic data'
 testing
 trackr-cnn
 TrackRCNN
 TrackR-CNN
 trackr-cnn-v2
'TrackR-CNN x BoxDist24'
'TrackR-CNN x BoxDist31'
'TrackR-CNN x BoxDist31 Video Outs'
 training
'Weekly Briefings'


##Import Libraries

In [3]:
# import some common libraries
from google.colab.patches import cv2_imshow
from sklearn.metrics import jaccard_score
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import datetime
import random
import json
import cv2
import csv
import os

# import some common pytorch utilities
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch

import pycocotools

##Set Directory

In [4]:
BASE_DIR = '/content/gdrive/My Drive/CMPT 419 720 Project/'
TESTING_DIR = os.path.join(BASE_DIR, 'testing')
OUTPUT_DIR = '{}/output'.format(BASE_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

#BoxDist31

##Network

Network Definition

In [5]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class BBoxDistNet(nn.Module):
  def __init__(self):
    super(BBoxDistNet, self).__init__()

    self.fc_net = nn.Sequential(
      nn.Linear(31, 18),
      nn.LayerNorm(18),
      nn.LeakyReLU(0.01, inplace=True),
      nn.Linear(18, 18),
      nn.LayerNorm(18),
      nn.LeakyReLU(0.01, inplace=True),
      nn.Linear(18, 3),
    )
  def forward(self, x):
    x = self.fc_net(x)
    return x

net = BBoxDistNet().cuda()

criterion = nn.L1Loss()

Load Pretrained Model

In [6]:
net = torch.load(os.path.join(OUTPUT_DIR, 'bbox31_807_1633_58.pt')).cuda()

#Testing

In [7]:
set_names = ['0002', '0006', '0007', '0008', '0010', '0013', '0014', '0016', '0018']

In [8]:
import math

def dist(xyz):
    x = xyz[0]
    y = xyz[1]
    z = xyz[2]
    return math.sqrt(x**2+y**2+z**2)

##predict displacement with forward euler
def dynamic_disp(v1, v2, a1, a2, t=0.1, h=0.001):
  ad = (a2-a1)/t
  d = 0
  v = v1
  a = a1
  for t in range(int(t/h)):
    a = a + h*ad
    v = v + h*a
    d += v*h + 0.5*a*h**2
  return d

##Load Test GT Labels (Expected Outputs)

In [9]:
def load_gt_dists(set_names):
  gt_len = 0
  annotations = {}
  kitti_anno_dir = os.path.join(TESTING_DIR, 'label_02')
  tracking_anno_dir = os.path.join(TESTING_DIR, 'tracking_data/segmentations')

  for set_name in set_names:
    kitti_anno_file = os.path.join(kitti_anno_dir, set_name+'.txt')
    tracking_anno_file = os.path.join(tracking_anno_dir, set_name+'.txt')
    records = {}

    tracking_anno = open(tracking_anno_file, 'r')
    tracking_lines = tracking_anno.readlines()
    line = tracking_lines[0]
    e = line.split(' ')
    height = int(e[3])
    width = int(e[4])

#Load Kitti Annotations
    kitti_anno = open(kitti_anno_file, 'r')
    kitti_lines = kitti_anno.readlines()
    for line in kitti_lines:
      e = line.split(' ')
      class_name = e[2]
      trunc = float(e[3])
      if (class_name != 'Car' and class_name != 'Pedestrian') or trunc > 0: #data exclusion
        continue
      frame = int(e[0])
      obj_id = int(e[1])
      if frame not in records.keys():
        records[frame] = {}
      if obj_id not in records[frame].keys():
        records[frame][obj_id] = {}
      records[frame][obj_id] = {
          'bbox': [float(e[6])/width, float(e[7])/height, float(e[8])/width, float(e[9])/height],
          'target': torch.from_numpy(np.array([float(e[13]), float(e[14]), float(e[15])])),
          'centre': [((float(e[6])+(float(e[8])-float(e[6]))/2)-(width/2))/width, ((float(e[7])+(float(e[9])-float(e[7]))/2)-(height/2))/height]
      }
      gt_len += 1
    annotations[set_name] = records
  print('GT has %d object instances' %(gt_len))
  return annotations

In [10]:
gt_obj_dists = load_gt_dists(set_names)

GT has 10846 object instances


##Load Track-RCNN Detections & Frame IMU Data (Inputs)

In [11]:
from pycocotools import mask as cocomask

def load_track_imu(set_names):
  pred_len = 0
  annotations = {}
  tracking_anno_dir = os.path.join(TESTING_DIR, 'tracking_data/segmentations')
  detection_anno_dir = os.path.join(TESTING_DIR, 'tracking_data/detections')
  gps_anno_dir = os.path.join(TESTING_DIR, 'oxts')

  for set_name in set_names:
    tracking_anno_file = os.path.join(tracking_anno_dir, set_name+'.txt')
    detection_anno_file = os.path.join(detection_anno_dir, set_name+'.txt')
    gps_anno_file = os.path.join(gps_anno_dir, set_name+'.txt')
    records = {}

#Load GPS Annotations
    gps_anno = open(gps_anno_file, 'r')
    gps_lines = gps_anno.readlines()
    gps = {}
    for frame, line in enumerate(gps_lines):
      e = line.split(' ')
      v = [float(e[8]), float(e[9]), float(e[10])]
      a = [float(e[11]), float(e[12]), float(e[13])]
      gps[frame] = {
          'v': v,
          'a': a,
      }
    
    tracking_anno = open(tracking_anno_file, 'r')
    tracking_lines = tracking_anno.readlines()
    for line in tracking_lines:
      e = line.split(' ')
      frame = int(e[0])
      if frame not in gps.keys():
        continue
      class_id = int(e[2])
      if class_id == 1:
        class_id = 0
      elif class_id == 2:
        class_id = 3
      obj_id = int(e[1])-1
      if frame not in records.keys():
        records[frame] = {}
        records[frame]['objs'] = {}
      height = int(e[3])
      width = int(e[4])
      records[frame]['height'] = height
      records[frame]['width'] = width
      records[frame]['v'] = gps[frame]['v']
      records[frame]['a'] = gps[frame]['a']
      records[frame]['frame'] = frame
      rle_obj = {
          'counts': e[5],
          'size': [height, width]
      }
      #bbox = cocomask.toBbox(rle_obj) #outputs XYWH
      if obj_id not in records[frame]['objs'].keys():
        records[frame]['objs'][obj_id] = {}
      #records[frame]['objs'][obj_id]['bbox'] = [bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]]
      records[frame]['objs'][obj_id]['rle'] = e[5].rstrip()
      records[frame]['objs'][obj_id]['class_id'] = class_id

    detection_anno = open(detection_anno_file, 'r')
    detection_lines = detection_anno.readlines()
    for line in detection_lines:
      e = line.split(' ')
      frame = int(e[0])
      confidence = float(e[5])
      if frame not in records.keys():
        continue
      bbox = [float(e[1]), float(e[2]), float(e[3]), float(e[4])]
      if 0.0 in bbox or width in bbox or height in bbox:# or confidence < 0.5 :
        continue
      rle = e[9].rstrip()
      for obj_id in records[frame]['objs'].keys():
        if records[frame]['objs'][obj_id]['rle'] == rle:
          records[frame]['objs'][obj_id]['bbox'] = bbox
          break
        pred_len += 1;

    annotations[set_name] = records
  
  print('TrackR-CNN has %d object instances' %(pred_len))
  return annotations


In [12]:
tracking_imu_data = load_track_imu(set_names)

TrackR-CNN has 169695 object instances


##Construct Model Inputs


In [13]:
def make_input(frame1, frame2, delta=1):
  height = frame1['height']
  width = frame1['width']
  img_center = [width/2, height/2]
  bbox1 = [frame1['bbox'][0]/width, frame1['bbox'][1]/height, frame1['bbox'][2]/width, frame1['bbox'][3]/height]
  bbox2 = [frame2['bbox'][0]/width, frame2['bbox'][1]/height, frame2['bbox'][2]/width, frame2['bbox'][3]/height]
  xd = -dynamic_disp(frame1['v'][1], frame2['v'][1], frame1['a'][1], frame2['a'][1], delta*0.1)
  yd = -dynamic_disp(frame1['v'][2], frame2['v'][2], frame1['a'][2], frame2['a'][2], delta*0.1)
  zd = dynamic_disp(frame1['v'][0], frame2['v'][0], frame1['a'][0], frame2['a'][0], delta*0.1)
  disp = [xd, yd, zd]
  bbox1 = [frame1['bbox'][0]/width, frame1['bbox'][1]/height, frame1['bbox'][2]/width, frame1['bbox'][3]/height]
  bbox2 = [frame2['bbox'][0]/width, frame2['bbox'][1]/height, frame2['bbox'][2]/width, frame2['bbox'][3]/height]
  w1 = bbox1[2] - bbox1[0]
  w2 = bbox2[2] - bbox2[0]
  h1 = bbox1[3] - bbox1[1]
  h2 = bbox2[3] - bbox2[1]
  a1 = w1*h1
  a2 = w2*h2
  c1 = [(frame1['bbox'][0]+(w1/2)-img_center[0])/(width/2), (frame1['bbox'][1]+(h1/2)-img_center[1])/(height/2)]
  c2 = [(frame2['bbox'][0]+(w2/2)-img_center[0])/(width/2), (frame2['bbox'][1]+(h2/2)-img_center[1])/(height/2)]
  class_id = frame1['class_id']
  id_vec = [0, 0, 0, 0, 0, 0, 0]
  id_vec[class_id] = 1
  input = torch.from_numpy(np.array(id_vec + bbox1 + bbox2 + disp + [w1, w2, h1, h2, a1, a2] + c1 + c2 + [w2/w1, h2/h1, a2/a1]).astype(np.float32)).cuda()
  return input

##Measure Accuracy

In [ ]:
#net = torch.load(os.path.join(OUTPUT_DIR, 'bbox31_807_1633_58.pt')).cuda()
net = torch.load(os.path.join(OUTPUT_DIR, 'bbox31_trackrcnn.pt')).cuda()

###Deprecated Code

In [ ]:
def pt_dists_2D(p0, pts):
  dists = []
  for pt in pts:
    dists.append(math.sqrt((p0[0]-pt[0])**2+(p0[1]-pt[1])**2))
  return dists

def bbox_overlap_ratio(bbox0, bboxs):
  ratios = []
  area0 = (bbox0[2]-bbox0[0]) * (bbox0[3]-bbox0[1])
  for bbox in bboxs:
    x_left = max(bbox0[0], bbox[0])
    y_top = max(bbox0[1], bbox[1])
    x_right = min(bbox0[2], bbox[2])
    y_bottom = min(bbox0[3], bbox[3])
    if x_right < x_left or y_bottom < y_top:
      iou = 0.0
      ratios.append(iou)
    else:
      intersection_area = (x_right - x_left) * (y_bottom - y_top)
      bb1_area = (bbox0[2] - bbox0[0]) * (bbox0[3] - bbox0[1])
      bb2_area = (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
      iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
      ratios.append(iou)

  return ratios

def align_input_to_gt(inputs, gts):
  input_cs = []
  gt_cs = []
  for input in inputs:
    numpy_input = input.detach().cpu().numpy()
    input_c = [numpy_input[26], numpy_input[27]]
    input_cs.append(input_c)
  for obj_id in gts.keys():
    gt_c = gts[obj_id]['centre']
    gt_cs.append(gt_c)
  
  dist_table = []
  for gt_c in gt_cs:
    dists = pt_dists_2D(gt_c, input_cs)
    dist_table.append(dists)
  dist_table = np.array(dist_table)

  alignments = [-1]*len(gts)

  input_mins = []
  gt_mins = []
  for i in range(len(inputs)):
    input_mins.append(np.argmin(dist_table[:, i]))
  for i in range(len(gts)):
    gt_mins.append(np.argmin(dist_table[i, :]))

  for i, gt_min in enumerate(gt_mins):
    if input_mins[gt_min] == i:
      alignments[i] = gt_min

  return alignments


def align_input_to_gt(inputs, gts):
  input_bboxs = []
  gt_bboxs = []
  for input in inputs:
    numpy_input = input.detach().cpu().numpy()
    input_bbox = [numpy_input[11], numpy_input[12], numpy_input[13], numpy_input[14]]
    input_bboxs.append(input_bbox)
  for obj_id in gts.keys():
    gt_bbox = gts[obj_id]['bbox']
    gt_bboxs.append(gt_bbox)
  
  ratio_table = []
  for gt_bbox in gt_bboxs:
    ratios = pt_dists_2D(gt_bbox, input_bboxs)
    ratio_table.append(ratios)
  ratio_table = np.array(ratio_table)

  alignments = [-1]*len(gts)

  input_maxs = []
  gt_maxs = []
  for i in range(len(inputs)):
    if np.max(ratio_table[:, i]) == 0:
      input_maxs.append(-2)
    else:
      input_maxs.append(np.argmax(ratio_table[:, i]))
  for i in range(len(gts)):
    if np.max(ratio_table[i, :]) == 0:
      gt_maxs.append(-2)
    else:
      gt_maxs.append(np.argmax(ratio_table[i, :]))
    
  for i, gt_max in enumerate(gt_maxs):
    if input_maxs[gt_max] == i:
      alignments[i] = gt_max

  return alignments

###Pred/Target Alignment

In [14]:
def bbox_overlap_ratio(bbox0, bboxs):
  ratios = []
  area0 = (bbox0[2]-bbox0[0]) * (bbox0[3]-bbox0[1])
  for bbox in bboxs:
    x_left = max(bbox0[0], bbox[0])
    y_top = max(bbox0[1], bbox[1])
    x_right = min(bbox0[2], bbox[2])
    y_bottom = min(bbox0[3], bbox[3])
    if x_right < x_left or y_bottom < y_top:
      iou = 0.0
      ratios.append(iou)
    else:
      intersection_area = (x_right - x_left) * (y_bottom - y_top)
      bb1_area = (bbox0[2] - bbox0[0]) * (bbox0[3] - bbox0[1])
      bb2_area = (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
      iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
      ratios.append(iou)

  return ratios

def align_input_to_gt(inputs, gts):
  input_bboxs = []
  gt_bboxs = []
  for input in inputs:
    numpy_input = input.detach().cpu().numpy()
    input_bbox = [numpy_input[11], numpy_input[12], numpy_input[13], numpy_input[14]]
    input_bboxs.append(input_bbox)
  for obj_id in gts.keys():
    gt_bbox = gts[obj_id]['bbox']
    gt_bboxs.append(gt_bbox)
  
  ratio_table = []
  for gt_bbox in gt_bboxs:
    ratios = bbox_overlap_ratio(gt_bbox, input_bboxs)
    ratio_table.append(ratios)
  ratio_table = np.array(ratio_table)

  alignments = [-1]*len(gts)

  gt_maxs = []
  for i in range(len(gts)):
    gt_maxs.append(np.argmax(ratio_table[i, :]))
  input_maxs = []
  for i in range(len(inputs)):
    input_maxs.append(np.argmax(ratio_table[:, i]))

  for me, want in enumerate(gt_maxs):
    if input_maxs[want] == me and ratio_table[me, want] > 0.0:
      alignments[me] = want

  return alignments

###Measure

In [35]:
frame_records = {}
output_records = []
target_records = []
n = 0
abs_errors = []
rel_errors = []
preds = []
dists = []
net.eval()
for set_name in set_names:
  frame_records[set_name] = {}
  prev_data = {}
  for frame in gt_obj_dists[set_name].keys():
    frame_records[set_name][frame] = {
        'bboxs': [],
        'preds': [],
        'dists': [],
    }
    if frame not in tracking_imu_data[set_name].keys():
      continue
    inputs = []
    for obj_id in tracking_imu_data[set_name][frame]['objs']:
      this_frame = tracking_imu_data[set_name][frame]
      if obj_id in prev_data.keys() and 'bbox' in this_frame['objs'][obj_id].keys():
        last_frame = prev_data[obj_id]
        if 'bbox' in last_frame['objs'][obj_id].keys():
          frame1 = {
              'v': last_frame['v'],
              'a': last_frame['a'],
              'bbox': last_frame['objs'][obj_id]['bbox'],
              'class_id': last_frame['objs'][obj_id]['class_id'],
              'height': last_frame['height'], 
              'width': last_frame['width'],
          }
          frame2 = {
              'v': this_frame['v'],
              'a': this_frame['a'],
              'bbox': this_frame['objs'][obj_id]['bbox'],
              'class_id': this_frame['objs'][obj_id]['class_id'],
          }
          delta = this_frame['frame'] - last_frame['frame']
          input = make_input(frame1, frame2, delta)
          inputs.append(input)
      prev_data[obj_id] = this_frame

    if len(inputs) == 0:
      continue

    outputs = []
    with torch.no_grad():
      for input in inputs:
        output = net(input).detach().cpu()
        outputs.append(output)
    
    frame_gt = gt_obj_dists[set_name][frame]

    alignment = align_input_to_gt(inputs, frame_gt)

    targets = []

    for obj_id in frame_gt.keys():
      targets.append(frame_gt[obj_id]['target'])
    
    for t, o in enumerate(alignment):
      if o == -1:
        continue
      output = outputs[o]
      target = targets[t]
      input = inputs[o]
      numpy_input = input.detach().cpu().numpy()
      input_bbox = [numpy_input[11], numpy_input[12], numpy_input[13], numpy_input[14]]
      frame_records[set_name][frame]['bboxs'].append(input_bbox)
      frame_records[set_name][frame]['preds'].append(dist(output))
      frame_records[set_name][frame]['dists'].append(dist(target))
      n += 1

In [27]:
print(n)

6702


#Video

In [49]:
def make_videos(set_name):
  img_dir = os.path.join(os.path.join(TESTING_DIR, 'image_02'), set_name)
  height = 374
  width = 1238
  img_center = [width/2, height/2]
  vid_writer = cv2.VideoWriter(filename=OUTPUT_DIR+"/track-rcnnxBoxdist31_"+set_name+".avi",  #Provide a file to write the video to
  fourcc=cv2.VideoWriter_fourcc('p', 'n', 'g', ' '),            #Use whichever codec works for you...
  fps=10,                                        #How many frames do you want to display per second in your video?
  frameSize=(width, height))   

  #prev data
  data = {}

  net.eval()
  with torch.no_grad():
      for frame in frame_records[set_name].keys():
        img_name = '000000'
        img_name = img_name[0:6-len(str(frame))] + str(frame)
        img = cv2.imread(os.path.join(img_dir, img_name+'.png'))
        print(img.shape)
        bboxes = frame_records[set_name][frame]['bboxs']
        preds = frame_records[set_name][frame]['preds']
        dists = frame_records[set_name][frame]['dists']
        for i, bbox in enumerate(bboxes):
          #draw boxes
          label = '%.f | %.2f' %(preds[i], dists[i])
          x1 = int(bbox[0]*width)
          x2 = int(bbox[2]*width)
          y1 = int(bbox[1]*height)
          y2 = int(bbox[3]*height)
          cv2.rectangle(img,(x1,y1),(x2,y2),(0,200,0),1)
          labelSize=cv2.getTextSize(label,cv2.FONT_HERSHEY_SIMPLEX,0.5,2)
          _x1 = x1
          _y1 = y1#+int(labelSize[0][1]/2)
          _x2 = _x1+labelSize[0][0]
          _y2 = y1-int(labelSize[0][1])
          cv2.rectangle(img,(_x1,_y1),(_x2,_y2),(0,200,0),cv2.FILLED)
          cv2.putText(img,label,(x1,y1),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,255),1)
          #write to video
        vid_writer.write(img)

  vid_writer.release()

In [50]:
make_videos('0018')

(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238, 3)
(374, 1238

In [24]:
vid_writer.release()

NameError: ignored